# Maestría en Ciencia de Datos, ITAM

**Curso de Optimización 2 2021-1,"Optimización Avanzada"
Prof. Erick Palacios Moreno**

* Equipo 5:  
>MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE 


## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING (RANDOM-RESTART)

* Distancia Euclideana.
> Dataset: National Traveling Salesman Problems, CANADA

Para ello, realizaremos una ejecución para un subgrupo del dataset con las primeras 17 ciudades con el objetivo de realizar una _test_ estandar en tiempo razonable.

______

**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

* Se integró el random_restart con un _for loop_ (default = 100), el cual repite el proceso de obtener la mejor ruta a partir de una de una _initial random solution_ con el objetivo de ampliar la búsqueda ya que detectamos como problemática que _Hill CLimbing_ se quedaba estacionado en un mínimo local. Esto se logró utilizando como paquetería de comparación OR Tools de Google (ver README para referencias).

* A continuación mostramos los resultados de nuestra reimplementación con Pool, para comparar resultados con nuestra antigua implementación de *Hill Climbing*, comparando tiempos entre ambos codigos. 

A continuación se muestra la descripción instancia utilizada:

In [4]:
%%bash
lscpu

Arquitectura:                        x86_64
modo(s) de operación de las CPUs:    32-bit, 64-bit
Orden de los bytes:                  Little Endian
Address sizes:                       39 bits physical, 48 bits virtual
CPU(s):                              12
Lista de la(s) CPU(s) en línea:      0-11
Hilo(s) de procesamiento por núcleo: 2
Núcleo(s) por «socket»:              6
«Socket(s)»                          1
Modo(s) NUMA:                        1
ID de fabricante:                    GenuineIntel
Familia de CPU:                      6
Modelo:                              158
Nombre del modelo:                   Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Revisión:                            13
CPU MHz:                             2926.443
CPU MHz máx.:                        4500.0000
CPU MHz mín.:                        800.0000
BogoMIPS:                            5199.98
Virtualización:                      VT-x
Caché L1d:                           192 KiB
Caché L1i:               

In [5]:
%%bash
sudo lshw -C memory

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


CalledProcessError: Command 'b'sudo lshw -C memory\n'' returned non-zero exit status 1.

In [2]:
import os
import time
import numpy as np
import pandas as pd
os.chdir("../")
from src.hill_cg.hill import best_solution
from src.hill_cg.hill_mul import parallel_hc